In [22]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
import random
import json
pylab.rcParams['figure.figsize'] = (8.0, 10.0)


In [23]:
dataDir='/Users/azakaria/Downloads'
#dataType='val2017'
annFile=f'/Users/azakaria/Downloads/nightowls_validation.json'

In [24]:
# initialize COCO api for instance annotations
coco=COCO(annFile)

loading annotations into memory...
Done (t=0.29s)
creating index...
index created!


In [25]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
cat_map = {}
for i,cat in enumerate(cats):
    cat_map[cat['id']] = i
nms=[cat['name'] for cat in cats]

In [26]:
catIds = coco.getCatIds()

In [27]:
negative_bbox_annotations = []
imgIds = coco.getImgIds()
def remove_negative_bbox_anns(anns):
    valid_anns = []
    negative_bbox = False
    for ann in anns:
        for bbox_val in ann['bbox']:
            if bbox_val <= 0:
                negative_bbox = True
        if negative_bbox == False:
            valid_anns.append(ann)
            #print(ann['bbox'])
        negative_bbox = False
    #for v in valid_anns:
        #print(v['bbox'])
    return valid_anns

from pathlib import Path
def coco_to_yolo_bb(x1, y1, w, h, image_w, image_h):
    #"bbox": [x,y,width,height], (COCO)
    #x_center y_center width height (YOLO)
    return [((2*x1 + w)/(2*image_w)) , ((2*y1 + h)/(2*image_h)), w/image_w, h/image_h]
def generate_labels(numImgs):
    """
    Some images:
    *Do not have annotations.
    *Have negative bboxes
    """
    # using with statement
    imgIdsPre = coco.getImgIds()
    imgIds = []
    random.shuffle(imgIds)
    #This should probably be refactored, but here we check that an image has an annotation
    i=0
    for imgId in imgIdsPre:
        annIds = coco.getAnnIds(imgIds = imgId, catIds=catIds)
        if(len(annIds) > 0):
            i += 1
            imgIds.append(imgId)
        if i >= 120:
            break

    #imgIds = imgIds[:numImgs]
    for imgId in imgIds:
        img = coco.loadImgs(imgId)[0] #If only 1 imgId, len() = 1
        label_dir = Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels')
        filename = img['file_name'].split('.')[0] + '.txt'
        with open(label_dir/filename, 'w') as file:
            annIds = coco.getAnnIds(imgIds = imgId, catIds=catIds)
            anns = coco.loadAnns(annIds)
            print(anns)
            print('-----------------------------')
            anns = remove_negative_bbox_anns(anns)
            for ann in anns:
                #filter out negative bb
                x_center, y_center, width, height = coco_to_yolo_bb(*ann['bbox'], img['width'], img['height'])
                #print('--------------')
                """
                print('filename')
                print("*ann['bbox']")
                print(*ann['bbox'])
                print("img['width'], img['height']")
                print(img['width'], img['height'])
                print('x_center,y_center,width,height')
                print(x_center,y_center,width,height)
                print('--------------')
                """
                #if(int(cat_map[ann["category_id"]]) > 2):
                    #print(cat_map[ann["category_id"]])
                file.write(f'{cat_map[ann["category_id"]]}\t{x_center}\t{y_center}\t{width}\t{height}\n')
            #print(f'imgId: {imgId}')
            #print(f'annIds: {annIds}')
            #print(f'anns: {anns}')
            #print(f'len(anns): {len(anns)}')
            #print(filename)
            #print(x_center)
    return


In [28]:
import os
import shutil
from pathlib import Path
import os.path

def copy_images_for_dataset_dir(src_image_dir, dest_image_dir, image_extension):
    """
    x number of labels are written to the labels dir by write_label_files(x)
    This function copies the corresponding images into the sibling images dir, completing the dataset folder for YOLO training
    """
    fs = os.listdir('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels')
    for f in fs:
        src = str(src_image_dir/f.split('.')[0]) + image_extension
        dest = str(dest_image_dir/ f.split('.')[0]) + image_extension
        #print(src)
        #print(dest)
        #print('----------------')
        if os.path.isfile(src):
            shutil.copyfile(src, dest)
            #print(f'{src} is a file')
        else:
            print(f'{src} is not a file')
    return


#rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/*; rm -rf /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/labels/*
#dataset/labels
generate_labels(120)
#dataset/images
copy_images_for_dataset_dir(Path('/Users/azakaria/Downloads/nightowls_validation'),Path('/Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/images/'), '.png')
#docker cp /Users/azakaria/Code/openmpf-yolo-training/nightowls/dataset/. 5dcf6e04dbba:/usr/src/datasets/nightowls_val
#ls nightowls/dataset/images | wc -l ; ls nightowls/dataset/labels | wc -l

[349, 227, 20, 41]
[645, 239, 21, 40]
[344, 231, 20, 41]
[654, 241, 24, 41]
[341, 229, 20, 41]
[659, 238, 25, 42]
[340, 229, 20, 41]
[661, 237, 26, 43]
[338, 229, 20, 41]
[667, 241, 21, 41]
[337, 228, 23, 42]
[668, 239, 21, 42]
[336, 227, 22, 43]
[671, 238, 21, 43]
[335, 226, 21, 43]
[675, 237, 21, 44]
[332, 226, 23, 46]
[679, 236, 21, 45]
[331, 225, 23, 47]
[682, 236, 21, 45]
[328, 225, 24, 48]
[686, 234, 21, 46]
[326, 226, 26, 48]
[689, 234, 21, 46]
[324, 226, 25, 47]
[694, 237, 21, 46]
[323, 229, 25, 48]
[698, 240, 22, 47]
[320, 231, 24, 49]
[702, 244, 22, 47]
[320, 232, 22, 51]
[707, 245, 22, 48]
[318, 232, 24, 52]
[711, 242, 23, 46]
[318, 230, 22, 52]
[718, 237, 20, 47]
[311, 228, 28, 53]
[723, 235, 21, 49]
[310, 227, 26, 54]
[509, 230, 79, 38]
[298, 231, 26, 57]
[512, 225, 55, 43]
[585, 246, 15, 30]
[294, 232, 28, 59]
[291, 233, 28, 58]
[286, 232, 32, 60]
[763, 242, 28, 30]
[283, 231, 28, 60]
[770, 242, 28, 45]
[282, 228, 27, 63]
[775, 242, 28, 61]
[276, 226, 30, 65]
[783, 242, 2